In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import sys
sys.path.append('..')
import dit_fast_attention
import os
file_path=os.path.abspath(dit_fast_attention.__file__)
dir_path=os.path.dirname(file_path)
os.chdir(dir_path)
from diffusers import DiTPipeline, DPMSolverMultistepScheduler
import argparse
import torch
from evaluation import test_latencies
from dit_fast_attention import transform_model_fast_attention
import numpy as np
import os
import time
import thop
from utils import calculate_flops
os.environ["https_proxy"]="http://10.10.20.100:1089"


In [ ]:
pipe = DiTPipeline.from_pretrained("facebook/DiT-XL-2-512", torch_dtype=torch.float16).to("cuda")
n_calib=8
n_steps=20
calib_x=torch.randint(0, 1000, (n_calib,),generator=torch.Generator().manual_seed(3)).to("cuda")


In [ ]:

# raw_latencies=test_latencies(pipe, n_steps,calib_x,bs=[1])
# print("Raw latencies",raw_latencies)
raw_flops=calculate_flops(pipe,calib_x[:1],n_steps)

In [ ]:
pipe = DiTPipeline.from_pretrained("facebook/DiT-XL-2-512", torch_dtype=torch.float16).to("cuda")
n_calib=8
n_steps=20
calib_x=torch.randint(0, 1000, (n_calib,),generator=torch.Generator().manual_seed(3)).to("cuda")
threshold=0.9
use_cache=True
pipe,calib_ssim=transform_model_fast_attention(pipe, n_steps=n_steps, n_calib=n_calib, calib_x=calib_x, threshold=threshold, window_size=[-64,64],use_cache=use_cache,seed=3)

latencies=test_latencies(pipe, n_steps,calib_x,bs=[1,8])
flops=calculate_flops(pipe,calib_x[:1],n_steps)

In [ ]:
from diffusers import Transformer2DModel, PixArtSigmaPipeline
transformer = Transformer2DModel.from_pretrained(
    "PixArt-alpha/PixArt-Sigma-XL-2-1024-MS",
    # "PixArt-alpha/PixArt-Sigma-XL-2-2K-MS",
    subfolder='transformer', 
    torch_dtype=torch.float16,
    use_safetensors=True,
)

pipe = PixArtSigmaPipeline.from_pretrained(
    'PixArt-alpha/pixart_sigma_sdxlvae_T5_diffusers',
    transformer=transformer,
    torch_dtype=torch.float16,
    use_safetensors=True,
).to("cuda")
pipe.config._name_or_path=transformer.config._name_or_path
n_calib=4
n_steps=20
# calib_x=torch.randint(0, 1000, (n_calib,),generator=torch.Generator().manual_seed(3)).to("cuda")
calib_x=["cat"]*n_calib


In [ ]:
flops=calculate_flops(pipe,calib_x[:1],n_steps)

In [ ]:

raw_latencies=test_latencies(pipe, n_steps,calib_x,bs=[1])
print("Raw latencies",raw_latencies)

In [ ]:
threshold=0.925
use_cache=True
pipe,calib_ssim=transform_model_fast_attention(pipe, n_steps=n_steps, n_calib=n_calib, calib_x=calib_x, threshold=threshold, window_size=[-256,256],use_cache=use_cache,seed=3)
# pipe,calib_ssim=transform_model_fast_attention(pipe, n_steps=n_steps, n_calib=n_calib, calib_x=calib_x, threshold=threshold, window_size=[-1024,1024],use_cache=use_cache,seed=3)

flops=calculate_flops(pipe,calib_x[:1],n_steps)

# latencies=test_latencies(pipe, n_steps,calib_x,bs=[1])
# print("Latencies",latencies)

In [ ]:
from diffusers import Transformer2DModel, PixArtSigmaPipeline
transformer = Transformer2DModel.from_pretrained(
    # "PixArt-alpha/PixArt-Sigma-XL-2-1024-MS",
    "PixArt-alpha/PixArt-Sigma-XL-2-2K-MS",
    subfolder='transformer', 
    torch_dtype=torch.float16,
    use_safetensors=True,
)

pipe = PixArtSigmaPipeline.from_pretrained(
    'PixArt-alpha/pixart_sigma_sdxlvae_T5_diffusers',
    transformer=transformer,
    torch_dtype=torch.float16,
    use_safetensors=True,
).to("cuda")
pipe.config._name_or_path=transformer.config._name_or_path
n_calib=2
n_steps=20
# calib_x=torch.randint(0, 1000, (n_calib,),generator=torch.Generator().manual_seed(3)).to("cuda")
calib_x=["cat"]*n_calib


In [ ]:
flops=calculate_flops(pipe,calib_x[:1],n_steps)

In [ ]:
threshold=0.9
use_cache=True
# pipe,calib_ssim=transform_model_fast_attention(pipe, n_steps=n_steps, n_calib=n_calib, calib_x=calib_x, threshold=threshold, window_size=[-256,256],use_cache=use_cache,seed=3)
pipe,calib_ssim=transform_model_fast_attention(pipe, n_steps=n_steps, n_calib=n_calib, calib_x=calib_x, threshold=threshold, window_size=[-1024,1024],use_cache=use_cache,seed=3)

flops=calculate_flops(pipe,calib_x[:1],n_steps)

# latencies=test_latencies(pipe, n_steps,calib_x,bs=[1])
# print("Latencies",latencies)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

sns.set_theme(style="whitegrid")
colors=sns.color_palette("Set2")

fig=plt.figure(figsize=(5,4))

# 假设有两组数据
# group1 1	0.897481743	1	0.762847116	1	0.495728137
# group2 0.871081781	0.82601481	0.673733854	0.610346823	0.338653802	0.302214254

# group1 = [1,0.897481743,1,0.836255068,1,0.701581777]
# group2 = [0.871081781,0.82601481,0.498965392,0.468643292,0.197997179,0.183244937]
group1=[1,0.897481743,1,0.762847116,1,0.495728137]
group2=[0.871081781,0.82601481,0.673733854,0.610346823,0.338653802,0.302214254]


raw_attn=group1[::2]
fast_attn=group1[1::2]
raw_other=group2[1::2]


# 创建 x 轴标签
x = np.arange(len(raw_attn))*1.5
labels = ['512x512\nraw', '512x512\nDitFastAttn', '1024x1024\nraw', '1024x1024\nDitFastAttn', '2048x2048\nraw', '2048x2048\nDitFastAttn']
labels = ['512x512', '1024x1024', '2048x2048']

# 绘制柱状图
fig, ax = plt.subplots()
ax.bar(x, raw_attn, 0.4,  label='Raw Attention', color=colors[0])

ax.bar(x+0.5, fast_attn, 0.4,  label='DiTFastAttn', color=colors[1])

offset=0.5
xs=np.concatenate([x,x+offset])
others=np.concatenate([raw_other,raw_other])
bar=ax.bar(xs, others, 0.4,  label='Others', color=colors[2])


# ax.bar(x+0.2, raw_other, 0.4,  label='Others', color=bar[0].get_facecolor())

# 设置 x 轴标签
ax.set_xticks(x+offset/2)
ax.set_xticklabels(labels)
# ax.set_xlabel('Category')
ax.set_ylabel('FLOPs Fraction')
ax.set_title('Fraction of FLOPs for different resolutions')
ax.legend(loc=3)

plt.savefig("plot/flops.pdf")
plt.show()

In [ ]:
import os
from PIL import Image
dir_raw="output/PixArt/PixArt-alpha_PixArt-Sigma-XL-2-1024-MS_steps20"
dir_fast="output/PixArt-alpha_PixArt-Sigma-XL-2-1024-MS_calib4_steps20_threshold0.95_window256_seqFalse"
for filename in os.listdir(dir_raw)[:30]:
    if filename.endswith(".jpg"):
        print(filename)
        file_raw=os.path.join(dir_raw,filename)
        file_fast=os.path.join(dir_fast,filename)
        # concat vertically and show
        img_raw=Image.open(file_raw)
        img_fast=Image.open(file_fast)
        img_concat=Image.new('RGB', (img_raw.width, img_raw.height + img_fast.height))
        img_concat.paste(img_raw, (0, 0))
        img_concat.paste(img_fast, (0, img_raw.height))
        # img_concat=Image.new('RGB', (img_raw.width + img_fast.width, img_raw.height))
        # img_concat.paste(img_raw, (0, 0))
        # img_concat.paste(img_fast, (img_raw.width, 0))
        # resize to 30%
        img_concat=img_concat.resize((int(img_concat.width*0.25),int(img_concat.height*0.25)))
        img_concat.show()
        
        